In [1]:
from polynomial_regression import *
from GPR import *
from random_forest import *
import ast

all_labels = ['height', 'phi', 'theta', 
        'impact site x', 'impact site y', 'impact site z', 
        'impact site r', 'impact site phi', 'impact site theta']

# deciding which features will be kept for the optimiation
features_to_keep = ['sqrt(angle_btw)', 'sqrt(crack len)', 'sqrt(dist btw frts)', 'angle_btw (unchanged)', 'dist btw frts * mean_kink', 'angle_btw + crack len', 'angle_btw + dist btw frts', 'crack len + dist btw frts']

labels_to_predict = ['height', 'impact site x', 'impact site y']
# labels_to_predict = ['impact site x']

model_types = ['poly2','GPR', 'RF']
# model_types = ['GPR']

# Generate some synthetic data for demonstration purposes
# full_dataset_pathname = "/Volumes/Jake_ssd/Paper_1_results_no_feature_engineering/dataset/New_Crack_Len_FULL_OG_dataframe_2023_11_16.csv"
full_dataset_pathname = "/Volumes/Jake_ssd/Paper_1_results_WITH_feature_engineering/dataset/feature_transformations_2023-11-16/height/HEIGHTALL_TRANSFORMED_FEATURES.csv"
df = pd.read_csv(full_dataset_pathname)
if(df.columns.__contains__('timestep_init')):
    df = df.drop('timestep_init', axis=1)

bayesian_opt_results_dir = '/Volumes/Jake_ssd/bayesian_optimization'



In [2]:
all_features_to_keep = {}
backward_feat_selection_results_path = '/Volumes/Jake_ssd/Paper_1_results_WITH_feature_engineering/results'
''' get the features to keep based on the best performing features on test sets in backward feature engineering outputs'''
''' only include more than 10 features though for now. '''
min_features = 10
max_features = 100
for label in labels_to_predict:
    all_features_to_keep[label] = {}
    for model_type in model_types:
        performances = pd.read_csv(backward_feat_selection_results_path + f'/{label}/{model_type}/performances/test_performances.csv')
        #only include rows that have less than 100 features and more than 10 features
        performances = performances.drop(performances[(performances['Unnamed: 0'] < 10) | (performances['Unnamed: 0'] > 100)].index)

        #calculate an average performance across all folds for each feature combination
        performances['average'] = performances[['fold0', 'fold1', 'fold2', 'fold3', 'fold4']].mean(axis=1)
        row_with_largest_average = performances['average'].idxmax() #get the row index that has the best performance across all folds

        #now get the features that were used for the best performing set of features
        features_kept = pd.read_csv(backward_feat_selection_results_path + f'/{label}/{model_type}/performances/features_kept.csv')
        best_feature_combination = features_kept.iloc[row_with_largest_average]['features remaining']
        all_features_to_keep[label][model_type] = ast.literal_eval(best_feature_combination)
                

In [3]:
for model_type in model_types:
    for label in labels_to_predict:
        print(f'\n $$$$$$$$$$$$$$$$$$$$$$$$$$ OPTIMIZING A {model_type} PREDICTING {label} $$$$$$$$$$$$$$$$$$$$$$$$$$ \n')
        features_to_keep = all_features_to_keep[label][model_type]
        saving_dir = f'{bayesian_opt_results_dir}/{label}'
        if(not os.path.exists(f'{saving_dir}/{model_type}')): os.makedirs(f'{saving_dir}/{model_type}')
        
        label_df = df.copy()[label]
        '''if there are defined features to keep, then only keep the defined ones. otherwise, just remove the labels for feat_df'''
        if(features_to_keep != 'ALL'):
            feat_df = df[features_to_keep]
        else:
            feat_df = df.drop(all_labels, axis=1)
        
        if(model_type =='GPR'):
            model = do_bayesian_optimization_GPR(feat_df, label_df, num_tries=100, saving_folder= saving_dir+'/GPR')
        elif(model_type == 'RF'):
            do_bayesian_optimization_RF(feat_df, label_df, num_tries=100, saving_folder= saving_dir+'/RF')
        elif(model_type == 'poly2'):
            do_bayesian_optimization_poly_reg(feat_df, label_df, num_tries=100, saving_folder= saving_dir+'/poly2')


 $$$$$$$$$$$$$$$$$$$$$$$$$$ OPTIMIZING A poly2 PREDICTING height $$$$$$$$$$$$$$$$$$$$$$$$$$ 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/jakehirst/miniconda3/envs/tfp_env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.037e-01, tolerance: 2.053e-02
  model = cd_fast.enet_coordinate_descent(


[CV 1/5] END elasticnet__alpha=0.013300153699806186, elasticnet__l1_ratio=0.6760795088769136;, score=-0.012 total time=   0.6s


/Users/jakehirst/miniconda3/envs/tfp_env/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.032e-01, tolerance: 2.188e-02
  model = cd_fast.enet_coordinate_descent(


[CV 2/5] END elasticnet__alpha=0.013300153699806186, elasticnet__l1_ratio=0.6760795088769136;, score=-0.810 total time=   0.8s


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=31, n_estimators=10000;, score=0.579 total time=   4.3s
[CV 2/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=31, n_estimators=10000;, score=0.556 total time=   4.2s
[CV 3/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=31, n_estimators=10000;, score=0.456 total time=   4.2s
[CV 4/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=31, n_estimators=10000;, score=0.521 total time=   4.2s
[CV 5/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=31, n_estimators=10000;, score=0.628 total time=   4.2s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=27, n_estimators=9999;, score=0.579 total time=   4.3s
[CV 2/5] END max_depth=3, max_features=3, min_samples_leaf=20

/Users/jakehirst/miniconda3/envs/tfp_env/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=29, n_estimators=10000;, score=0.579 total time=   4.2s
[CV 2/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=29, n_estimators=10000;, score=0.556 total time=   4.5s
[CV 3/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=29, n_estimators=10000;, score=0.456 total time=   4.2s
[CV 4/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=29, n_estimators=10000;, score=0.521 total time=   4.3s
[CV 5/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=29, n_estimators=10000;, score=0.628 total time=   4.2s


/Users/jakehirst/miniconda3/envs/tfp_env/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=32, n_estimators=10000;, score=0.579 total time=   4.2s
[CV 2/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=32, n_estimators=10000;, score=0.556 total time=   4.2s
[CV 3/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=32, n_estimators=10000;, score=0.456 total time=   4.2s
[CV 4/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=32, n_estimators=10000;, score=0.521 total time=   4.2s
[CV 5/5] END max_depth=3, max_features=3, min_samples_leaf=20, min_samples_split=32, n_estimators=10000;, score=0.628 total time=   4.2s

Best parameters found:  OrderedDict([('max_depth', 3), ('max_features', 3), ('min_samples_leaf', 20), ('min_samples_split', 27), ('n_estimators', 9999)])
Train R^2 score: 0.6426081933946978
Test R^2 score: 0.48747108227249736
